#Lending Club Scoring Code

Part 3 of 3: This code should take the model from part 2 and score a group of new accounts for investment. This code mainly focuses on appropriately cleaning the variables in order to score. Tests throughout will make sure their are not dramatic data differences between the build/validation sample compared to newly scored accounts.

###Import necessary Python Packages

In [1]:
import pandas as pd
import time
import datetime
import numpy as np 
import statsmodels.formula.api as sm
import json

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

###Create Dictionary to track passes/failures

In [3]:
testDict = {}

###1) Create DF of potential investments from downloaded CSVs

In [26]:
prospects = pd.io.parsers.read_csv('/Users/tphoran/Downloads/primaryMarketNotes_browseNotes_1-RETAIL.csv', sep=',', index_col=False)
prospects = prospects.set_index(['id'])
prospects.head()

,member_id,loan_amnt,funded_amnt,term,int_rate,exp_default_rate,service_fee_rate,installment,grade,sub_grade,...,num_il_tl,mo_sin_old_il_acct,num_actv_rev_tl,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,total_rev_hi_lim,num_rev_tl_bal_gt_0,num_op_rev_tl,tot_coll_amt,effective_int_rate
id,,,,,,,,,,,,,,,,,,,,,
61390816,65509604,5000,4925,36,6.89,1.42,0.78,154.14,A,A3,...,4,161,8,82,12,26900,8,9,0,6.84
61522012,65640818,12000,11875,36,7.89,2.00,0.78,375.43,A,A5,...,2,146,5,194,4,30000,5,6,0,7.81
61531914,65650720,5000,4900,36,6.24,1.20,0.78,152.66,A,A2,...,2,124,9,267,6,25100,9,16,0,6.20
61438891,65557619,18000,17650,36,7.89,2.00,0.78,563.15,A,A5,...,7,142,5,217,9,150500,5,8,0,7.81
61481328,65600118,20000,19575,36,5.32,1.00,0.78,602.30,A,A1,...,2,131,6,314,27,40600,6,8,0,5.29


####Test to see if columns have been added since code was initially built or datatype has changed

In [27]:
#Expected columns as of 2015-10-04
expected = [u'member_id', u'loan_amnt', u'funded_amnt', u'term', u'int_rate', u'exp_default_rate', u'service_fee_rate', u'installment', u'grade', u'sub_grade', u'emp_title', u'emp_length', u'home_ownership', u'annual_inc', u'is_inc_v', u'accept_d', u'exp_d', u'list_d', u'credit_pull_d', u'review_status_d', u'review_status', u'url', u'desc', u'purpose', u'title', u'zip_code', u'addr_state', u'msa', u'acc_now_delinq', u'acc_open_past_24mths', u'bc_open_to_buy', u'percent_bc_gt_75', u'bc_util', u'dti', u'delinq_2yrs', u'delinq_amnt', u'earliest_cr_line', u'fico_range_low', u'fico_range_high', u'inq_last_6mths', u'mths_since_last_delinq', u'mths_since_last_record', u'mths_since_recent_inq', u'mths_since_recent_revol_delinq', u'mths_since_recent_bc', u'mort_acc', u'open_acc', u'pub_rec', u'total_bal_ex_mort', u'revol_bal', u'revol_util', u'total_bc_limit', u'total_acc', u'total_il_high_credit_limit', u'num_rev_accts', u'mths_since_recent_bc_dlq', u'initial_list_status', u'ils_exp_d', u'pub_rec_bankruptcies', u'num_accts_ever_120_pd', u'chargeoff_within_12_mths', u'collections_12_mths_ex_med', u'tax_liens', u'mths_since_last_major_derog', u'num_sats', u'num_tl_op_past_12m', u'mo_sin_rcnt_tl', u'tot_hi_cred_lim', u'tot_cur_bal', u'avg_cur_bal', u'num_bc_tl', u'num_actv_bc_tl', u'num_bc_sats', u'pct_tl_nvr_dlq', u'num_tl_90g_dpd_24m', u'num_tl_30dpd', u'num_tl_120dpd_2m', u'num_il_tl', u'mo_sin_old_il_acct', u'num_actv_rev_tl', u'mo_sin_old_rev_tl_op', u'mo_sin_rcnt_rev_tl_op', u'total_rev_hi_lim', u'num_rev_tl_bal_gt_0', u'num_op_rev_tl', u'tot_coll_amt', u'effective_int_rate']

current = prospects.columns

for i in current:
    if i not in expected:
        testDict['1) Create DF - Columns'] = 'Fail'
else:
    testDict['1) Create DF - Columns'] = 'Pass'

In [28]:
#Expected data types as of 2015-10-04
expectedDtype = {'pub_rec_bankruptcies': 'int64', 'inq_last_6mths': 'int64', 'num_sats': 'int64', 'acc_open_past_24mths': 'int64', 'emp_length': 'object', 'emp_title': 'object', 'pub_rec': 'int64', 'title': 'object', 'exp_default_rate': 'float64', 'earliest_cr_line': 'object', 'mths_since_recent_bc': 'int64', 'avg_cur_bal': 'int64', 'review_status_d': 'object', 'total_bal_ex_mort': 'int64', 'term': 'int64', 'mths_since_last_major_derog': 'object', 'desc': 'object', 'num_op_rev_tl': 'int64', 'num_actv_rev_tl': 'int64', 'installment': 'float64', 'mo_sin_old_il_acct': 'object', 'num_rev_accts': 'int64', 'home_ownership': 'object', 'zip_code': 'object', 'mths_since_last_delinq': 'object', 'num_bc_tl': 'int64', 'pct_tl_nvr_dlq': 'int64', 'total_rev_hi_lim': 'int64', 'ils_exp_d': 'float64', 'revol_util': 'float64', 'total_bc_limit': 'int64', 'delinq_2yrs': 'int64', 'num_accts_ever_120_pd': 'int64', 'num_tl_120dpd_2m': 'object', 'accept_d': 'object', 'percent_bc_gt_75': 'float64', 'mo_sin_rcnt_rev_tl_op': 'int64', 'num_tl_90g_dpd_24m': 'int64', 'mths_since_recent_inq': 'object', 'member_id': 'int64', 'tot_hi_cred_lim': 'int64', 'loan_amnt': 'float64', 'delinq_amnt': 'float64', 'collections_12_mths_ex_med': 'int64', 'tax_liens': 'int64', 'grade': 'object', 'review_status': 'object', 'annual_inc': 'float64', 'num_il_tl': 'int64', 'bc_util': 'float64', 'initial_list_status': 'object', 'num_tl_30dpd': 'int64', 'is_inc_v': 'object', 'msa': 'float64', 'num_bc_sats': 'int64', 'tot_coll_amt': 'int64', 'sub_grade': 'object', 'mths_since_last_record': 'object', 'dti': 'float64', 'revol_bal': 'float64', 'mths_since_recent_revol_delinq': 'object', 'total_il_high_credit_limit': 'int64', 'int_rate': 'float64', 'mo_sin_rcnt_tl': 'int64', 'addr_state': 'object', 'credit_pull_d': 'object', 'service_fee_rate': 'float64', 'tot_cur_bal': 'int64', 'num_rev_tl_bal_gt_0': 'int64', 'fico_range_low': 'int64', 'total_acc': 'int64', 'list_d': 'object', 'fico_range_high': 'int64', 'exp_d': 'object', 'mo_sin_old_rev_tl_op': 'int64', 'num_actv_bc_tl': 'int64', 'mort_acc': 'int64', 'open_acc': 'int64', 'mths_since_recent_bc_dlq': 'object', 'bc_open_to_buy': 'int64', 'purpose': 'object', 'acc_now_delinq': 'int64', 'chargeoff_within_12_mths': 'int64', 'url': 'object', 'effective_int_rate': 'float64', 'funded_amnt': 'float64', 'num_tl_op_past_12m': 'int64'}

for i in expectedDtype:
    if str(prospects[i].dtype) != expectedDtype[i]:
        testDict['1) Create DF - dType'] = 'Fail'
else:
    testDict['1) Create DF - dType'] = 'Pass'   

###2) Clean data in same way as build when possible

In [29]:
# Info on to_datetimes http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html
# Converting to a Date in Words - Year in Numbers format (Jan-2015)
def timeBetweenDatesM(timeZero, timeCompare):
    timeZeroF = pd.to_datetime(timeZero,format="%b-%Y")
    timeCompareF = pd.to_datetime(timeCompare,format="%b-%Y")
    return (timeCompareF - timeZeroF).astype('timedelta64[M]')

# Add column for today: https://docs.python.org/2/library/datetime.html#strftime-strptime-behavior
prospects['today'] = datetime.date.today().strftime("%b-%Y")

#Fix earliest_cr_line varialbe
prospects['earliest_cr_line'] = pd.to_datetime(prospects['earliest_cr_line'])
prospects['earliest_cr_line'] = prospects['earliest_cr_line'].apply(lambda x: x.strftime("%b-%Y"))

# Apply definition
prospects['earliest_cr_line_R'] = timeBetweenDatesM(prospects['today'], prospects['earliest_cr_line'])

#Add a 0 loan_age field
prospects['loan_age'] = 0

In [30]:
#Clean revol_util and int_rate as we had in build
prospects['revol_util'] = prospects.revol_util.astype('float')/100
prospects['int_rate'] = prospects.int_rate.astype('float')/100

In [31]:
#Drop columns either irrelevant to scoring or not available when model was built
dropVariables = [
'service_fee_rate',
'exp_default_rate',
'emp_title',
'accept_d',
'exp_d',
'list_d',
'credit_pull_d',
'review_status_d',
'review_status',
'url',
'desc',
'title',
'msa',
'acc_now_delinq',
'acc_open_past_24mths',
'bc_open_to_buy',
'percent_bc_gt_75',
'bc_util',
'delinq_amnt',
'earliest_cr_line',
'mths_since_recent_inq',
'mths_since_recent_revol_delinq',
'mths_since_recent_bc',
'mort_acc',
'total_bal_ex_mort',
'total_bc_limit',
'total_il_high_credit_limit',
'num_rev_accts',
'mths_since_recent_bc_dlq',
'ils_exp_d',
'num_accts_ever_120_pd',
'chargeoff_within_12_mths',
'pub_rec_bankruptcies',
'mo_sin_rcnt_tl',
'tot_hi_cred_lim',
'tot_cur_bal',
'avg_cur_bal',
'num_bc_tl',
'num_actv_bc_tl',
'num_bc_sats',
'pct_tl_nvr_dlq',
'num_tl_90g_dpd_24m',
'num_tl_30dpd',
'num_tl_120dpd_2m',
'num_il_tl',
'mo_sin_old_il_acct',
'num_actv_rev_tl',
'mo_sin_old_rev_tl_op',
'mo_sin_rcnt_rev_tl_op',
'total_rev_hi_lim',
'num_rev_tl_bal_gt_0',
'num_op_rev_tl',
'tot_coll_amt',
'effective_int_rate',
'today',
'num_tl_op_past_12m',
'num_sats'
]

#Dataframe with only the relevant variables
prospects = prospects.select(lambda f: f not in dropVariables,axis=1)

In [34]:
# Fix object variables that shouls be numeric
print prospects['mths_since_last_delinq'].describe(include = 'all')
print 
print prospects['mths_since_last_record'].describe(include = 'all')
print 
print prospects['mths_since_last_major_derog'].describe(include = 'all')

prospects['mths_since_last_delinq'] = prospects['mths_since_last_delinq'].apply(lambda x: None if x == ' ' else x)
prospects['mths_since_last_delinq'] = prospects.mths_since_last_delinq.astype('float')

prospects['mths_since_last_record'] = prospects['mths_since_last_record'].apply(lambda x: None if x == ' ' else x)
prospects['mths_since_last_record'] = prospects.mths_since_last_record.astype('float')

prospects['mths_since_last_major_derog'] = prospects['mths_since_last_major_derog'].apply(lambda x: None if x == 'null' else x)
prospects['mths_since_last_major_derog'] = prospects.mths_since_last_major_derog.astype('float')

print
print prospects['mths_since_last_delinq'].describe(include = 'all')
print 
print prospects['mths_since_last_record'].describe(include = 'all')
print 
print prospects['mths_since_last_major_derog'].describe(include = 'all')

count     741
unique     80
top          
freq      378
Name: mths_since_last_delinq, dtype: object

count     741
unique     75
top          
freq      574
Name: mths_since_last_record, dtype: object

count      741
unique      88
top       null
freq       528
Name: mths_since_last_major_derog, dtype: object

count    363.000000
mean      34.250689
std       22.281734
min        1.000000
25%       16.000000
50%       30.000000
75%       52.000000
max       83.000000
Name: mths_since_last_delinq, dtype: float64

count    167.000000
mean      65.011976
std       21.735538
min        4.000000
25%       54.000000
50%       65.000000
75%       79.500000
max      112.000000
Name: mths_since_last_record, dtype: float64

count    213.000000
mean      48.910798
std       26.567936
min        1.000000
25%       30.000000
50%       51.000000
75%       66.000000
max      146.000000
Name: mths_since_last_major_derog, dtype: float64


In [37]:
#Convert term to a string
print prospects['term'].value_counts()

def termClean(term):
    if term == 36:
        return '36 months'
    elif term == 60:
        return '60 months'

prospects['term'] = prospects.apply(lambda row: termClean(row['term']), axis=1)

print
print prospects['term'].value_counts()

36    483
60    258
dtype: int64

36 months    483
60 months    258
dtype: int64


In [39]:
#Fix Income Verification
print prospects['is_inc_v'].value_counts()

def verificationStatusClean(is_inc_v):
    if is_inc_v == 'Not Verified':
        return 'not verified'
    elif is_inc_v == 'Source Verified':
        return 'VERIFIED - income source'
    elif is_inc_v == 'Verified':
        return 'VERIFIED - income'

prospects['verification_status'] = prospects.apply(lambda row: verificationStatusClean(row['is_inc_v']), axis=1)

print
print prospects['verification_status'].value_counts()

Not Verified       579
Source Verified    141
Verified            21
dtype: int64

not verified                579
VERIFIED - income source    141
VERIFIED - income            21
dtype: int64


In [41]:
#Fix purpose variable
print prospects['purpose'].value_counts()

def purposeClean(purpose):
    if purpose == 'Debt consolidation':
        return 'debt_consolidation'
    elif purpose == 'Credit card refinancing':
        return 'credit_card'
    elif purpose == 'Home improvement':
        return 'home_improvement'
    elif purpose == 'Other':
        return 'other'
    elif purpose == 'Major purchase':
        return 'major_purchase'
    elif purpose == 'Business':
        return 'small_business' 
    elif purpose == 'Medical expenses':
        return 'medical'
    elif purpose == 'Vacation':
        return 'vacation'
    elif purpose == 'Car financing':
        return 'car'
    elif purpose == 'Moving and relocation':
        return 'moving'    

prospects['purpose'] = prospects.apply(lambda row: purposeClean(row['purpose']), axis=1)

print
print prospects['purpose'].value_counts()

Debt consolidation         395
Credit card refinancing    255
Home improvement            50
Other                       19
Major purchase              12
Car financing                4
Business                     2
Medical expenses             2
Green loan                   1
Vacation                     1
dtype: int64

debt_consolidation    395
credit_card           255
home_improvement       50
other                  19
major_purchase         12
car                     4
small_business          2
medical                 2
vacation                1
dtype: int64


In [40]:
print prospects['purpose'].value_counts()

Debt consolidation         395
Credit card refinancing    255
Home improvement            50
Other                       19
Major purchase              12
Car financing                4
Business                     2
Medical expenses             2
Green loan                   1
Vacation                     1
dtype: int64


In [ ]:
print prospects['term'].value_counts()

def termClean(term):
    if term == 36:
        return '36 months'
    elif term == 60:
        return '60 months'

prospects['term'] = prospects.apply(lambda row: termClean(row['term']), axis=1)

print
print prospects['term'].value_counts()

In [32]:
prospects.columns

Index([u'member_id', u'loan_amnt', u'funded_amnt', u'term', u'int_rate', u'installment', u'grade', u'sub_grade', u'emp_length', u'home_ownership', u'annual_inc', u'is_inc_v', u'purpose', u'zip_code', u'addr_state', u'dti', u'delinq_2yrs', u'fico_range_low', u'fico_range_high', u'inq_last_6mths', u'mths_since_last_delinq', u'mths_since_last_record', u'open_acc', u'pub_rec', u'revol_bal', u'revol_util', u'total_acc', u'initial_list_status', u'collections_12_mths_ex_med', u'tax_liens', u'mths_since_last_major_derog', u'earliest_cr_line_R', u'loan_age'], dtype='object')

In [ ]:
#Check Expected columns
expectedFields = ['member_id', 'loan_amnt', 'funded_amnt' 'term', 'int_rate', 
                  'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 
                  'verification_status','purpose', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 
                  'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 
                  'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 
                  'initial_list_status', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 
                  'earliest_cr_line_R', 'loan_age']



####Test variable range for fixed int_rate and revol_util

In [18]:
def varRangeCheck(series, acceptable_range, expected_mean, expected_25_quantile, expected_75_quantile):
    if series.mean() > (expected_mean*(1+acceptable_range)) or series.mean() < (expected_mean*(1-acceptable_range)) or series.quantile(0.25) > (expected_25_quantile*(1+acceptable_range)) or series.quantile(0.25) < (expected_25_quantile*(1-acceptable_range)) or series.quantile(0.75) > (expected_75_quantile*(1+acceptable_range)) or series.quantile(0.75) < (expected_75_quantile*(1-acceptable_range)):
        return 'Fail'
    else:
        return 'Pass'

#Set acceptable variable variance
acceptable_range = 0.05    

#Expected metrics as of 2015-09-03 from build
expected_mean_int_rate = 0.138
expected_25_int_rate = 0.1099
expected_75_int_rate = 0.1649

testDict['2) Data Fix - int_rate Range'] = varRangeCheck(prospects['int_rate'], acceptable_range, expected_mean_int_rate, expected_25_int_rate, expected_75_int_rate)

#Expected metrics as of 2015-09-03 from build
expected_mean_revol_util = 0.561
expected_25_revol_util = 0.393
expected_75_revol_util = 0.747

testDict['2) Data Fix - revol_util Range'] = varRangeCheck(prospects['revol_util'], acceptable_range, expected_mean_revol_util, expected_25_revol_util, expected_75_revol_util)

In [22]:
print prospects['int_rate'].mean()
print prospects['int_rate'].quantile(0.25)
print prospects['int_rate'].quantile(0.75)

print prospects['int_rate'].mean()
print prospects['int_rate'].quantile(0.25)
print prospects['int_rate'].quantile(0.75)

0.0968862348178
0.0789
0.1099


In [19]:
print testDict

{'1) Create DF - dType': 'Pass', '2) Data Fix - revol_util Range': 'Fail', '2) Data Fix - int_rate Range': 'Fail', '1) Create DF - Columns': 'Pass'}
